In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import numpy as np
import pickle
import haversine as hs
from haversine import Unit


In [ ]:
input_data_folder_base = Path("../results/")
input_data_folder = Path.cwd() / input_data_folder_base
overall_collection_file_name = "overall_collection_2023.p"
key_parameter_file_name = "key_parameter_2023.p"
print(f"Reading raw pandas files from \n    {input_data_folder}")

output_data_folder = input_data_folder
print(f"Writing merged pandas file to \n    {output_data_folder}")

In [ ]:
df_results = pd.read_pickle(input_data_folder / overall_collection_file_name)
df_results.head(3)

In [ ]:

def dist(lat_0, lon_0, lat_1, lon_1):
    return hs.haversine((lat_0, lon_0), (lat_1, lon_1), unit=Unit.METERS)

df_results["distance(m)"] = 0
df_results["old_lat"] = df_results["lat_est"].shift(-1)
df_results["old_lon"] = df_results["lon_est"].shift(-1)
for index, row in df_results.iterrows():
    if index != df_results.index[0]:
        df_results.at[index, "distance(m)"] = dist(row['lat_est'], row['lon_est'], row['old_lat'], row['old_lon'])


In [ ]:
ax=df_results[df_results["distance(m)"]<2]["distance(m)"].hist()
ax.set_title("Speed level [m/s]")

In [ ]:
df_key_param = df_results[df_results.meas_running].reset_index()\
        .groupby('corresponding_meas_file')\
            .agg(start=('datetime_AMT', 'min'),
                 end=('datetime_AMT', 'max'),
                 experiment_location = ('experiment_location', 'first'),
                 position = ('position', 'first'),
                 lat_est_mean=('lat_est', 'mean'),
                 lon_est_mean=('lon_est', 'mean'),
                 hdop_max=('hdop', 'max'),
                 nsats_min=('nsats', 'min'),
                 CO2_min=('CO2(ppm)', 'min'),
                 CO2_max=('CO2(ppm)', 'max'),
                 humidity_in_mean=('humidity in(rH)', 'mean'),
                 humidity_out_mean=('humidity out(rH)', 'mean'),
                 airtemp_in_mean=('airtemp in(degreeC)', 'mean'),
                 PAR_min=('PAR(umol m-2s-1)', 'min'),
                 PAR_max=('PAR(umol m-2s-1)', 'max'),
                 airtemp_out_mean=('airtemp out(degreeC)', 'mean'))

result_file = output_data_folder / Path(key_parameter_file_name) 
print(f"    ... saving to \n        {result_file}")
df_key_param.to_pickle(result_file)